<br>

<div style ="width: 100%;">
<table style ="width: 100%;background-color:float:center;">
<tr>

</tr>
    
<tr>
        
<td>
<div style="font-size: 21px;Margin-left:-10px;">
<b>Lógica Computacional TP1<b>
</div>
    <br>
    <br>
<div style="font-size: 16px;">
    
Realizado por:

Miguel Gonçalves a90416
 
João Nogueira a87973
</div>
</td>
<td >
<div style="display:inline-block; float:right;Margin-right:0px;Margin-top:20px;">
<img src="https://www.ecum.uminho.pt/SiteAssets/ECUM-PT.png" width="260" height="120">
<div>
</td>
</tr>
</table>
</div>

### Exercicio 1



1. Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
  1. Cada reunião ocupa uma sala (enumeradas 1...S) durante um “slot” (tempo,dia). Assume-se os dias enumerados 1..D e, em cada dia, os tempos enumerados 1..T.
  2. Cada reunião tem associado um projeto (enumerados 1..P) e um conjunto de participantes. Os diferentes colaboradores são enumerados 1..C.
  3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
  4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  50\% do total de colaboradores do projeto. A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema). 

**Limitações**

1. Cada colaborador só estar em uma reunião de cada vez:

$$\forall_{d < D} \cdot \forall_{t < T} \cdot \forall_{c < C} \sum_{s < S,\,p < P} x_{s,t,d,p,c} \leq 1$$

**Obrigações**

1. Cada projeto realiza $n$ de reuniões semanais (como o lider está presente em todas, o lider tem que fazer $n$ reuniões semanais:

$$\forall_{p < P} \cdot \quad \sum_{s < S,\,t < T,\,d < D,\,l < L} x_{s,t,d,p,l} = N,\ onde\ l\ é\ o\ líder\ do\ projeto\ p $$
<br>

2. No mínimo 50% dos colaboradores têm de aparecer no projeto:


$$\forall_{d < D} \cdot \forall_{t < T} \cdot \forall_{s < S} \cdot \forall_{p < P} \cdot \quad \sum_{c < C} x_{s,t,d,p,c} \geq \frac{C}{2}  + C\ mod\ 2\ \bigvee \sum_{c < C} x_{s,t,d,p,c} = 0 $$

3. O líder têm de aparecer em todas as reuniões do seu projeto (se um colaborador for a uma reunião o lider tem que estar la):

$$\forall_{s < S} \cdot \forall_{t < T} \cdot \forall_{d < D} \cdot \forall_{p < P} \cdot \forall_{c < C} \quad ( x_{s,t,d,p,c} == 1 ) \rightarrow (x_{s,t,d,p,l} == 1) $$

4. Cada sala tem alocado no máximo um projeto de cada vez:

$$\forall_{d < D} \cdot  \forall_{t < T} \cdot  \forall_{s < S} \sum_{p < P} x_{s,t,d,p,l} \leq 1 $$

### Versão com o z3

In [ ]:
!pip install z3-solver

In [ ]:
from z3 import *

S,T,D,P,C = 2,2,2,2,4
  
def constroiHorario(reunioesSemanais, colaboradoresProjeto, lider , disponibilidade):


    horario = Solver()
    X = {}
    
    for s in range(1,S+1):  #Para cada sala enumerada 1...S
        for t in range(1,T+1):  #Para cada tempo enumerado 1..T
            for d in range(1,D+1):  #Para cada dia enumerado 1..D
                for p in range(1,P+1):  #Para cada projeto enumerado 1..P
                    for c in range(1,C+1):  #Para cada colaborador enumerado 1..C
                      
                        #Se pertencer ao projeto tem de estar presente ou não (ou é 1 ou 0)
                        if c in colaboradoresProjeto[p]:
                            X[s,t,d,p,c] = Int(str(s) + '_' + str(t) + '_' + str(d) + '_' + str(p) + '_' + str(c))
                            horario.add (X[s,t,d,p,c] >= 0 , X[s,t,d,p,c] <=1)
                        #Se não pertencer ao projeto não está presente
                        else:
                            X[s,t,d,p,c] = Int(str(s) + '_' + str(t) + '_' + str(d) + '_' + str(p) + '_' + str(c))
                            horario.add (X[s,t,d,p,c] == 0)
    

    # Limitação 1 - Cada colaborador só estar em uma reunião de cada vez
    
    for t in range(1,T+1):
        for d in range(1,D+1):
            for c in range(1,C+1):
                horario.add(sum([X[s,t,d,p,c] for s in range(1,S+1) for p in range (1,P+1)]) <= 1 )

    
    # Obrigação 1 - Cada projeto realiza n de reuniões semanais
    
    for p in range(1,P+1):
        horario.add(sum([X[s,t,d,p,lider[p]] for s in range (1,S+1) for d in range (1,D+1) for t in range (1,T+1)]) == reunioesSemanais[p] )
    

    # Obrigação 2 - No mínimo 50% dos colaboradores têm de aparecer no projeto
    # Se a soma dos colaboradores for 0 significa que não existirá reunião, caso contrário , esta soma terá de ser maior que 50% dos colaboradores
    # Caso o número de colaboradores seja ímpar , juntamos o resto da divisão inteira de C por 2.

    for s in range(1,S+1):
        for t in range(1,T+1):
            for d in range(1,D+1):
                for p in range(1,P+1):
                    horario.add(Or(sum([X[s,t,d,p,c] for c in colaboradoresProjeto[p] ]) == 0  , sum([X[s,t,d,p,c] for c in colaboradoresProjeto[p] ]) >= ((len(colaboradoresProjeto[p])/2) + (len(colaboradoresProjeto[p])%2))))
                    
    
    
    # Obrigação 3 - O líder têm de aparecer em todas as reuniões do seu projeto
    
    for s in range(1,S+1):
        for t in range(1,T+1):       
            for d in range(1,D+1):
                for p in range(1,P+1):
                    for c in range(1,C+1):
                        horario.add(Implies(X[s,t,d,p,c]==1 ,X[s,t,d,p,lider[p]] ==1))
        
        
    # Obrigação 4 - Cada sala tem alocado no máximo um projeto de cada vez

    for t in range(1,T+1):
        for d in range(1,D+1):
            for s in range(1,S+1):
                    horario.add(Sum([X[s,t,d,p,lider[p]] for p in range(1,P+1)]) <= 1)
    
    
    # Nao pode ir a reuniao (disponibilidade)
    
    for c in range(1,C+1):
        for t in range(1,T+1):
            for d in range(1,D+1):
                if (t,d) not in disponibilidade[c]:
                    horario.add(And([(X[s,t,d,p,c] == 0) for s in range (1,S+1) for p in range(1,P+1)]))
    
  
    r = horario.check()
    print(r)
    
    if r == sat:
        m = horario.model()
        for d in range (1,D+1):
            print ("Dia" , d,":")
            for p in range (1,P+1):
                for t in range (1,T+1):
                    for s in range (1,S+1):
                        for c in range (1,C+1):
                            if m[X[s,t,d,p,c]] == 1:
                                print ("Projeto",p,"Tempo",t,"Sala",s,"Colaborador",c)

In [ ]:
S,T,D,P,C = 2,2,2,2,5
reunioesSemanais = {1:1,2:3}
colaboradoresProjeto = {1:[1,2,4,5] , 2:[3,5]}
lider = {1:4,2:5}
disponibilidade = {1: [(1,1)] , 2: [] , 3: [(1,2),(2,2)] , 4: [(1,1)] , 5: [(1,1),(2,1) ,(1,2),(2,2)]}
constroiHorario (reunioesSemanais, colaboradoresProjeto, lider , disponibilidade)

sat
Dia 1 :
Projeto 1 Tempo 1 Sala 2 Colaborador 4
Projeto 1 Tempo 1 Sala 2 Colaborador 5
Projeto 2 Tempo 2 Sala 1 Colaborador 5
Dia 2 :
Projeto 2 Tempo 1 Sala 1 Colaborador 5
Projeto 2 Tempo 2 Sala 1 Colaborador 5


In [ ]:
S,T,D,P,C = 2,4,5,6,8
#40 possiveis reunioes 2x4x5
reunioesSemanais = {1:4,2:6,3:10,4:5,5:5,6:10}
colaboradoresProjeto = {1:[1,3,4,8],2:[2,8],3:[3,7],4:[4],5:[5,7],6:[6,8,5]}
lider = {1:1, 2:2,3:3,4:4,5:5,6:6}

disponibilidade = {}

for i in range(1,9):
    disponibilidade[i] =[(1,1), (1,2), (1,3), (1,4), (1,5), (2,1), (2,2), (2,3), (2,4), (2,5), (3,1), (3,2), (3,3), (3,4), (3,5), (4,1), (4,2), (4,3), (4,4), (4,5)]

constroiHorario (reunioesSemanais, colaboradoresProjeto, lider , disponibilidade)

sat
Dia 1 :
Projeto 3 Tempo 2 Sala 2 Colaborador 3
Projeto 3 Tempo 3 Sala 2 Colaborador 3
Projeto 3 Tempo 4 Sala 2 Colaborador 3
Projeto 4 Tempo 1 Sala 2 Colaborador 4
Projeto 5 Tempo 4 Sala 1 Colaborador 5
Projeto 6 Tempo 1 Sala 1 Colaborador 5
Projeto 6 Tempo 1 Sala 1 Colaborador 6
Projeto 6 Tempo 2 Sala 1 Colaborador 5
Projeto 6 Tempo 2 Sala 1 Colaborador 6
Projeto 6 Tempo 3 Sala 1 Colaborador 6
Projeto 6 Tempo 3 Sala 1 Colaborador 8
Dia 2 :
Projeto 1 Tempo 2 Sala 2 Colaborador 1
Projeto 1 Tempo 2 Sala 2 Colaborador 4
Projeto 1 Tempo 2 Sala 2 Colaborador 8
Projeto 3 Tempo 1 Sala 1 Colaborador 3
Projeto 3 Tempo 3 Sala 2 Colaborador 3
Projeto 4 Tempo 4 Sala 2 Colaborador 4
Projeto 5 Tempo 1 Sala 2 Colaborador 5
Projeto 5 Tempo 4 Sala 1 Colaborador 5
Projeto 6 Tempo 2 Sala 1 Colaborador 5
Projeto 6 Tempo 2 Sala 1 Colaborador 6
Projeto 6 Tempo 3 Sala 1 Colaborador 6
Projeto 6 Tempo 3 Sala 1 Colaborador 8
Dia 3 :
Projeto 2 Tempo 2 Sala 2 Colaborador 2
Projeto 2 Tempo 4 Sala 1 Colaborador

### Versão SCIP

In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

S,T,D,P,C = 0,0,0,0,0

def horario(reunioes, colaboradores, lideres, disponibilidade):

  solver = pywraplp.Solver.CreateSolver('SCIP')
  x = {}

  for s in range(1,S+1):  #Para cada sala enumerada 1...S
     for t in range(1,T+1): #Para cada tempo enumerado 1..T
       for d in range(1,D+1): #Para cada dia enumerado 1..D
         for p in range(1,P+1): #Para cada projeto enumerado 1..P
           for c in range(1,C+1): #Para cada colaborador enumerado 1..C

             #Se pertencer ao projeto tem de estar presente ou não
                      if c in colaboradoresProjeto[p]:
                        x[s,t,d,p,c] = solver.BoolVar("x[%d,%d,%d,%d,%d]" % (s,t,d,p,c))
                        solver.Add(x[s,t,d,p,c] >= 0, x[s,t,d,p,c] <= 1)
                      #Se não pertencer ao projeto não está presente
                      else:
                        x[s,t,d,p,c] = solver.BoolVar("x[%d,%d,%d,%d,%d]" % (s,t,d,p,c))
                        solver.Add(x[s,t,d,p,c] == 0)


  # Limitação 1 - O colaborador não pode estar em mais que uma reuniao ao mesmo tempo (Limitação nº1)
  for t in range(1,T+1):
        for d in range(1,D+1):
            for c in range(1,C+1):
                horario.Add(sum([X[s,t,d,p,c] for s in range(1,S+1) for p in range (1,P+1)]) <= 1 )


  # Obrigação 1 - Cada projeto realiza  n  de reuniões semanais
  for p in range(P+1):
    solver.Add(sum([x[s,t,d,p,lider[p]] for s in range (1,S+1) for d in range (1,D+1) for t in range (1,T+1)]) == reunioesSemanais[p] )
    

  # Obrigação 2 - No mínimo 50% dos colaboradores têm de aparecer no projeto
  for s in range(1,S+1):
    for t in range(1,T+1):
      for d in range(1,D+1):
        for p in range(1,P+1):
          solver.Add(Or(sum([x[s,t,d,p,c] for c in colaboradoresProjeto[p] ]) == 0 , sum([x[s,t,d,p,c] for c in colaboradoresProjeto[p] ]) >= ((len(colaboradoresProjeto[p])/2) + (len(colaboradoresProjeto[p])%2))))


  # Obrigação 3 - O líder têm de aparecer em todas as reuniões do seu projeto
  for s in range(1,S+1):
    for t in range(1,T+1):       
      for d in range(1,D+1):
        for p in range(1,P+1):
          for c in range(1,C+1):
            solver.Add(Implies(x[s,t,d,p,c]==1 ,x[s,t,d,p,lider[p]] ==1))


  # Obrigação 4 - Cada sala tem alocado no máximo um projeto de cada vez
  for t in range(1,T+1):
    for d in range(1,D+1):
      for s in range(1,S+1):
        solver.Add(sum([x[s,t,d,p,lider[p]] for p in range(1,P+1)]) <= 1)


  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
    for d in range (1,D+1):
      print ("Dia" , d,":")
      for p in range (1,P+1):
        for t in range (1,T+1):
          for s in range (1,S+1):
            for c in range (1,C+1):
              if round(x[s,t,d,p,c].solution_value()) == 1:
                print ("Projeto",p,"Tempo",t,"Sala",s,"Colaborador",c)

In [ ]:
S,T,D,P,C = 2,2,2,2,5
reunioesSemanais = {1:1,2:3}
colaboradoresProjeto = {1:[1,2,4,5] , 2:[3,5]}
lider = {1:4,2:5}
disponibilidade = {1: [(1,1)] , 2: [] , 3: [(1,2),(2,2)] , 4: [(1,1)] , 5: [(1,1),(2,1) ,(1,2),(2,2)]}
horario(reunioesSemanais, colaboradoresProjeto, lider , disponibilidade)